In [1]:
import os
import math

from estnltk import Text
from estnltk.converters import json_to_text
from estnltk.converters import text_to_json

from estnltk.layer.layer import Layer
from estnltk import EnvelopingBaseSpan
from estnltk.layer_operations import flatten

from nervaluate import Evaluator
import pandas as pd

from modules.preprocessing_protocols import preprocess_text

In [2]:
files_not_working = ['J2rva_Tyri_V22tsa_id22177_1911a.json', \
                     'J2rva_Tyri_V22tsa_id18538_1894a.json', \
                     'J2rva_Tyri_V22tsa_id22155_1911a.json', \
                     'Saare_Kihelkonna_Kotlandi_id18845_1865a.json', \
                     'P2rnu_Halliste_Abja_id257_1844a.json', \
                     'Saare_Kaarma_Loona_id7575_1899a.json', \
                     'J2rva_Tyri_V22tsa_id22266_1913a.json', \
                     'J2rva_Tyri_V22tsa_id22178_1912a.json']

In [4]:
def results_by_subdistribution(results_json):

    correct = results_json['strict']['correct']        
    actual = results_json['strict']['actual']
    possible = results_json['strict']['possible']
        
    precision = (correct / actual)
    recall = (correct / possible)
    f1 = 2 * ((precision * recall) / (precision + recall))
    
    return precision, recall, f1

In [5]:
files = []
with open(os.path.join('..', 'data', 'divided_corpus.txt'), 'r', encoding='UTF-8') as f:
    lines = f.readlines()

for line in lines:
    filename, subdistribution = line.strip().split(':')
    files.append(filename)

In [6]:
def extract_results(model_dir, files):    
    gold_ner = []
    test_ner = []

    for file in files:
        appendable_gold_ner = []
        appendable_test_ner = []

        if not file.endswith(".json") or file in files_not_working:
            continue
        else:
            with open(os.path.join(model_dir, file), 'r', encoding='UTF-8') as f_test, \
                 open(os.path.join('..', 'data', 'vallakohtufailid-json-flattened', file), 'r', encoding='UTF-8') as f_gold:
                    test_import = json_to_text(f_test.read())
                    gold_import = json_to_text(f_gold.read())

                    for i in range(len(gold_import['gold_ner'])):
                        ner = gold_import['gold_ner'][i]
                        label = ner.nertag
                        start = int(ner.start)
                        end = int(ner.end)
                        appendable_gold_ner.append({"label": label, "start": start, "end": end})

                    for i in range(len(test_import['flat_ner'])):
                        ner = test_import['flat_ner'][i]
                        label = ner.nertag[0]
                        start = int(ner.start)
                        end = int(ner.end)
                        appendable_test_ner.append({"label": label, "start": start, "end": end})

        gold_ner.append(appendable_gold_ner)
        test_ner.append(appendable_test_ner)
    evaluator = Evaluator(gold_ner, test_ner, tags=['ORG', 'PER', 'MISC', 'LOC', 'LOC_ORG'])
    results, results_per_tag = evaluator.evaluate()
    print("Tulemuste ammutamine on lõpetatud.")

    return results, results_per_tag

In [7]:
def multivalued_entity_dict(path, leave_out_O, as_percentages):
    me_dict = dict()
    texts_dict = dict()
    
    # Read in all filenames from path
    for file in os.listdir(path):
        with open(os.path.join(path, file), 'r', encoding='UTF-8') as in_f:
            texts_dict[file] = json_to_text(in_f.read())
    
    # Create me_dict, which contains all named entities
    for file in texts_dict.keys():
        text = texts_dict[file]
        
        for entity in text.flat_wordner:
            named_entity = entity.text
            nertag = entity.nertag[0]
            labels = dict()
            if entity.text in me_dict.keys():
                if nertag in me_dict[named_entity]:
                    labels = me_dict[named_entity]
                    labels[nertag] = labels[nertag] + 1
                else:
                    labels = me_dict[named_entity]
                    labels[nertag] = 1
                    me_dict[named_entity] = labels
            else:
                me_dict[named_entity] = {nertag: 1}
    
    # Remove the ones that only have one label (including 'O')
    
    # If leave_out_O is True, leave out all named entites that
    # have other labels as well as the 'O' label 
    delete_keys = set()
    for key, value in me_dict.items():
        if len(value.items()) == 1:
            delete_keys.add(key)
        elif leave_out_O and 'O' in value.keys():
            delete_keys.add(key)

    for key in delete_keys:
        del me_dict[key]

    # If as_precentages is True, display all values in me_dict
    # as percentages instead of counts
    if as_percentages:
        for key, value in me_dict.items():
            all_counts = 0
            for ne, count in value.items():
                all_counts += count

            for ne, count in value.items():
                percentage = round((count / all_counts) * 100, 3)
                value[ne] = percentage

    return me_dict

In [8]:
def change_multivalued_entities_in_file(text, me_dict, threshold, threshold_O = 100):
    
    for i in range(len(text.flat_wordner)):
        entity = text.flat_wordner[i].text
        nertag = text.flat_wordner[i].nertag[0]
        
        # If the named entity is in the dict, check if it needs to be
        # changed due to the threshold
        
        if entity in me_dict:
            for key, value in me_dict[entity].items():
                before = text.flat_wordner[i].nertag[0]
                if key != 'O' and value > threshold and nertag != key:
                    text.flat_wordner[i].nertag = key
                    print(f'Muudatus sõnes {entity}: {before} -> {key}')
                elif key == 'O' and value > threshold_O and nertag != key:
                    text.flat_wordner[i].nertag = key
                    print(f'Muudatus sõnes {entity}: {before} -> {key}')
    return text

In [9]:
def convert_wordner_to_ner(text):
    snt_labels = [label.nertag[0] for label in text.flat_wordner]
    
    text.pop_layer('flat_ner')
    text = preprocess_text(text)    
    nerlayer = Layer(name='ner', attributes=['nertag'], text_object=text, enveloping='words')
    
    entity_spans = []
    entity_type = None
    
    for span, label in zip(text.words, snt_labels):
        if entity_type is None:
            entity_type = label[2:]
        if label == "O":
            if entity_spans:
                nerlayer.add_annotation(EnvelopingBaseSpan(entity_spans),
                                        **{['nertag'][0]: entity_type})
                entity_spans = []
            continue
        if label[0] == "B" or entity_type != label[2:]:
            if entity_spans:
                nerlayer.add_annotation(EnvelopingBaseSpan(entity_spans),
                                        **{['nertag'][0]: entity_type})
                entity_spans = []
        entity_type = label[2:]
        entity_spans.append(span.base_span)
        
    text.add_layer(nerlayer)
    
    flat_ner = flatten(text['ner'], 'flat_ner')
    text.add_layer(flat_ner)
    
    removed_layers = ['compound_tokens', 'tokens', 'words']
    for layer in removed_layers:
        text.pop_layer(layer)
    return text

In [10]:
path_to_read = os.path.join('models',
                    'model_morph_with_lemmas_and_sentences_and_gazetteer_and_global_features', 
                    'model_gaz_loc_variants', 
                    'vallakohtufailid-trained-nertagger')

path_to_write = os.path.join('models',
                    'model_morph_with_lemmas_and_sentences_and_gazetteer_and_global_features', 
                    'model_gaz_loc_variants_me', 
                    'vallakohtufailid-trained-nertagger')

---

In [11]:
setups = [[True, 85], # Leave out O-s, if majority above 85% -> change
          [True, 90], # Leave out O-s, if majority above 90% -> change 
          [True, 95], # Leave out O-s, if majority above 95% -> change
          [False, 85, 95], # Don't leave out O-s, if majority of !O above 85% -> change, also change O if above 95%
          [False, 85, 99], # Don't leave out O-s, if majority of !O above 85% -> change, also change O if above 99%
          [False, 90, 95], # Don't leave out O-s, if majority of !O above 90% -> change, also change O if above 95%
          [False, 90, 99], # Don't leave out O-s, if majority of !O above 90% -> change, also change O if above 99%
          [False, 95, 95], # Don't leave out O-s, if majority of !O above 95% -> change, also change O if above 95%
          [False, 95, 99]] # Don't leave out O-s, if majority of !O above 95% -> change, also change O if above 99%

In [14]:
all_results = dict()

for setup in setups:
    leave_out_O = setup[0]
    threshold = int(setup[1])
    try:
        threshold_O = setup[2]
    except:
        threshold_O = 100
        
    me_dict = multivalued_entity_dict(path_to_read, leave_out_O, as_percentages = True)
    
    for file in os.listdir(path_to_read):
        if file.endswith('json'):
            with open(os.path.join(path_to_read, file), 'r', encoding='UTF-8') as in_f:
                text = json_to_text(in_f.read())
            
            text = change_multivalued_entities_in_file(text, me_dict, threshold, threshold_O)
            text = convert_wordner_to_ner(text)
            text_to_json(text, file=os.path.join(path_to_write, file))
            
    all_results[f'{str(leave_out_O)}, {str(threshold)}, {str(threshold_O)}'] = (extract_results(path_to_write, files))

Muudatus sõnes Miina: I-PER -> B-PER
Muudatus sõnes Kändler: B-PER -> I-PER
Muudatus sõnes Maddis: I-PER -> B-PER
Muudatus sõnes Ludwi: B-PER -> I-PER
Muudatus sõnes Tio: B-LOC_ORG -> B-PER
Muudatus sõnes Aadu: I-PER -> B-PER
Muudatus sõnes Aadu: I-PER -> B-PER
Muudatus sõnes Mart: I-PER -> B-PER
Muudatus sõnes Ader: B-PER -> I-PER
Muudatus sõnes Isak: I-PER -> B-PER
Muudatus sõnes Hinrek: I-PER -> B-PER
Muudatus sõnes Hinrek: I-PER -> B-PER
Muudatus sõnes Ummus: B-PER -> I-PER
Muudatus sõnes Johanson: B-PER -> I-PER
Muudatus sõnes Püll: B-PER -> I-PER
Muudatus sõnes Käit: B-PER -> I-PER
Muudatus sõnes Marri: I-PER -> B-PER
Muudatus sõnes Anne: I-PER -> B-PER
Muudatus sõnes Jaagup: B-LOC_ORG -> B-PER
Muudatus sõnes Jaagup: B-LOC_ORG -> B-PER
Muudatus sõnes Pukki: B-LOC_ORG -> I-PER
Muudatus sõnes Keusi: B-PER -> B-LOC_ORG
Muudatus sõnes Peterson: B-PER -> I-PER
Muudatus sõnes Willemsohn: B-PER -> I-PER
Muudatus sõnes Kaarli: I-PER -> B-PER
Muudatus sõnes Ado: I-PER -> B-PER
Muudatus sõ

Muudatus sõnes Willemsohn: B-PER -> I-PER
Muudatus sõnes Kaarli: I-PER -> B-PER
Muudatus sõnes Ado: I-PER -> B-PER
Muudatus sõnes Tedder: B-PER -> I-PER
Muudatus sõnes Ilwes: B-PER -> I-PER
Muudatus sõnes Jürgenson: B-PER -> I-PER
Muudatus sõnes Kaarel: B-LOC_ORG -> B-PER
Muudatus sõnes Otto: I-PER -> B-PER
Muudatus sõnes Hindrek: I-PER -> B-PER
Muudatus sõnes Hindrek: I-PER -> B-PER
Muudatus sõnes Hindrek: I-PER -> B-PER
Muudatus sõnes Karli: I-PER -> B-PER
Muudatus sõnes Walk: B-PER -> I-PER
Muudatus sõnes Tõnnis: I-PER -> B-PER
Muudatus sõnes Tõnnis: I-PER -> B-PER
Muudatus sõnes Rein: I-PER -> B-PER
Muudatus sõnes Tomson: B-PER -> I-PER
Muudatus sõnes Madis: B-LOC_ORG -> B-PER
Muudatus sõnes Mihkel: B-LOC_ORG -> B-PER
Muudatus sõnes Hans: I-PER -> B-PER
Muudatus sõnes Johann: I-PER -> B-PER
Muudatus sõnes Mänd: B-LOC_ORG -> I-PER
Muudatus sõnes Marie: B-LOC_ORG -> B-PER
Muudatus sõnes Nõmm: B-PER -> I-PER
Muudatus sõnes Nõmm: B-PER -> I-PER
Muudatus sõnes Lauk: B-PER -> I-PER
Muuda

Muudatus sõnes Jüri: O -> B-PER
Muudatus sõnes Jüri: I-PER -> B-PER
Muudatus sõnes ja: I-ORG -> O
Muudatus sõnes Jüri: I-ORG -> B-PER
Muudatus sõnes ja: I-ORG -> O
Muudatus sõnes Jüri: I-ORG -> B-PER
Muudatus sõnes Jüri: B-LOC_ORG -> B-PER
Muudatus sõnes küüni: I-LOC -> O
Muudatus sõnes hobune: I-PER -> O
Muudatus sõnes Ader: B-PER -> I-PER
Muudatus sõnes K: I-PER -> B-PER
Muudatus sõnes K: O -> B-PER
Muudatus sõnes Sults: O -> I-PER
Muudatus sõnes Jürri: O -> B-PER
Muudatus sõnes ja: I-ORG -> O
Muudatus sõnes ja: I-ORG -> O
Muudatus sõnes ja: I-ORG -> O
Muudatus sõnes ja: I-ORG -> O
Muudatus sõnes Isak: I-PER -> B-PER
Muudatus sõnes Hinrek: I-PER -> B-PER
Muudatus sõnes Hinrek: I-PER -> B-PER
Muudatus sõnes ja: I-ORG -> O
Muudatus sõnes toas: I-LOC -> O
Muudatus sõnes ja: I-LOC -> O
Muudatus sõnes ja: I-LOC -> O
Muudatus sõnes Ummus: B-PER -> I-PER
Muudatus sõnes Johanson: B-PER -> I-PER
Muudatus sõnes ja: I-LOC -> O
Muudatus sõnes ja: I-LOC -> O
Muudatus sõnes ja: I-LOC -> O
Muudatus

Muudatus sõnes Matwei: I-PER -> B-PER
Muudatus sõnes Jüri: I-PER -> B-PER
Muudatus sõnes Miina: I-PER -> B-PER
Muudatus sõnes Johani: I-PER -> B-PER
Muudatus sõnes Karli: I-PER -> B-PER
Muudatus sõnes a: I-PER -> O
Muudatus sõnes Tikk: B-PER -> I-PER
Muudatus sõnes [: I-PER -> O
Muudatus sõnes A: O -> B-PER
Muudatus sõnes -: I-LOC_ORG -> O
Muudatus sõnes -: I-LOC_ORG -> O
Muudatus sõnes Kass: O -> I-PER
Muudatus sõnes Kütt: B-PER -> I-PER
Muudatus sõnes Tomkin: B-PER -> I-PER
Muudatus sõnes aial: B-PER -> O
Muudatus sõnes -: I-LOC_ORG -> O
Muudatus sõnes -: I-PER -> O
Muudatus sõnes -: I-ORG -> O
Muudatus sõnes Nikke: O -> I-PER
Muudatus sõnes Andrejew: B-PER -> I-PER
Muudatus sõnes Peterson: B-PER -> I-PER
Muudatus sõnes Aru: O -> I-PER
Muudatus sõnes A: I-MISC -> B-PER
Muudatus sõnes P: O -> B-PER
Muudatus sõnes Feodor: O -> B-PER
Muudatus sõnes Iwanow: B-PER -> I-PER
Muudatus sõnes Polakse: I-PER -> B-LOC_ORG
Muudatus sõnes Polakse: I-PER -> B-LOC_ORG
Muudatus sõnes Laar: B-PER -> I

Muudatus sõnes Jürri: O -> B-PER
Muudatus sõnes Kiik: O -> I-PER
Muudatus sõnes P: O -> B-PER
Muudatus sõnes A: O -> B-PER
Muudatus sõnes Jürri: O -> B-PER
Muudatus sõnes Aid: O -> I-PER
Muudatus sõnes Pak: O -> I-PER
Muudatus sõnes wallast: O -> I-LOC_ORG
Muudatus sõnes Liina: O -> B-PER
Muudatus sõnes Zuping: I-PER -> B-PER
Muudatus sõnes (: I-LOC_ORG -> O
Muudatus sõnes Kikkas: B-LOC_ORG -> I-PER
Muudatus sõnes Peterson: B-PER -> I-PER
Muudatus sõnes Trofim: I-PER -> B-PER
Muudatus sõnes Purge: B-PER -> I-PER
Muudatus sõnes Ruus: B-PER -> I-PER
Muudatus sõnes -: I-LOC -> O
Muudatus sõnes -: I-LOC -> O
Muudatus sõnes -: I-LOC -> O
Muudatus sõnes -: I-LOC_ORG -> O
Muudatus sõnes Suurest: B-LOC_ORG -> B-LOC
Muudatus sõnes wõi: I-PER -> O
Muudatus sõnes -: I-LOC_ORG -> O
Muudatus sõnes -: I-LOC -> O
Muudatus sõnes -: I-LOC -> O
Muudatus sõnes -: I-LOC -> O
Muudatus sõnes -: I-LOC -> O
Muudatus sõnes -: I-ORG -> O
Muudatus sõnes Kohtu: I-ORG -> O
Muudatus sõnes -: I-LOC_ORG -> O
Muudatus

...

Muudatus sõnes Jürri: O -> B-PER
Muudatus sõnes P: O -> B-PER
Muudatus sõnes A: O -> B-PER
Muudatus sõnes Jürri: O -> B-PER
Muudatus sõnes Aid: O -> I-PER
Muudatus sõnes (: I-LOC_ORG -> O
Muudatus sõnes Kikkas: B-LOC_ORG -> I-PER
Muudatus sõnes Trofim: I-PER -> B-PER
Muudatus sõnes Purge: B-PER -> I-PER
Muudatus sõnes Ruus: B-PER -> I-PER
Muudatus sõnes -: I-LOC -> O
Muudatus sõnes -: I-LOC -> O
Muudatus sõnes -: I-LOC -> O
Muudatus sõnes -: I-LOC_ORG -> O
Muudatus sõnes Suurest: B-LOC_ORG -> B-LOC
Muudatus sõnes wõi: I-PER -> O
Muudatus sõnes -: I-LOC_ORG -> O
Muudatus sõnes -: I-LOC -> O
Muudatus sõnes -: I-LOC -> O
Muudatus sõnes -: I-LOC -> O
Muudatus sõnes -: I-LOC -> O
Muudatus sõnes -: I-ORG -> O
Muudatus sõnes Kohtu: I-ORG -> O
Muudatus sõnes -: I-LOC_ORG -> O
Muudatus sõnes Kõppu: B-LOC_ORG -> I-LOC_ORG
Muudatus sõnes -: I-PER -> O
Muudatus sõnes -: I-LOC_ORG -> O
Muudatus sõnes -: I-LOC_ORG -> O
Muudatus sõnes -: I-LOC_ORG -> O
Muudatus sõnes -: I-LOC_ORG -> O
Muudatus sõnes 

Muudatus sõnes P: I-PER -> B-PER
Muudatus sõnes :: I-PER -> O
Muudatus sõnes Josepi: O -> B-PER
Muudatus sõnes Niediko: B-PER -> I-PER
Muudatus sõnes Waldmann: B-PER -> I-PER
Muudatus sõnes Waldmann: B-PER -> I-PER
Muudatus sõnes Edro: B-PER -> I-PER
Muudatus sõnes Jürri: O -> B-PER
Muudatus sõnes Jürri: I-PER -> B-PER
Muudatus sõnes Kristjaan: B-LOC_ORG -> B-PER
Muudatus sõnes Jaan: I-PER -> B-PER
Muudatus sõnes Märt: I-PER -> B-PER
Muudatus sõnes Jaan: I-PER -> B-PER
Muudatus sõnes P: I-PER -> B-PER
Muudatus sõnes P: I-PER -> B-PER
Muudatus sõnes P: I-PER -> B-PER
Muudatus sõnes P: I-PER -> B-PER
Muudatus sõnes P: I-PER -> B-PER
Muudatus sõnes P: I-PER -> B-PER
Muudatus sõnes Jakub: B-LOC_ORG -> B-PER
Muudatus sõnes P: I-PER -> B-PER
Muudatus sõnes P: I-PER -> B-PER
Muudatus sõnes P: I-PER -> B-PER
Muudatus sõnes P: I-PER -> B-PER
Muudatus sõnes P: I-PER -> B-PER
Muudatus sõnes Mihkel: I-PER -> B-PER
Muudatus sõnes Juhan: I-PER -> B-PER
Muudatus sõnes Abram: I-PER -> B-PER
Muudatus s

Muudatus sõnes kohut: I-ORG -> O
Muudatus sõnes -: I-LOC_ORG -> O
Muudatus sõnes Ado: I-PER -> B-PER
Muudatus sõnes tütar: I-PER -> O
Muudatus sõnes -: I-LOC_ORG -> O
Muudatus sõnes s: I-MISC -> O
Muudatus sõnes Weber: O -> I-PER
Muudatus sõnes kohtule: I-ORG -> O
Muudatus sõnes a: I-LOC_ORG -> O
Muudatus sõnes ): I-LOC_ORG -> O
Muudatus sõnes Kaarel: B-LOC_ORG -> B-PER
Muudatus sõnes Otto: I-PER -> B-PER
Muudatus sõnes Hindrek: I-PER -> B-PER
Muudatus sõnes Hindrek: I-PER -> B-PER
Muudatus sõnes Hindrek: I-PER -> B-PER
Muudatus sõnes Jürri: O -> B-PER
Muudatus sõnes -: I-MISC -> O
Muudatus sõnes -: I-PER -> O
Muudatus sõnes rendi: B-LOC_ORG -> O
Muudatus sõnes a: I-PER -> O
Muudatus sõnes Märtsil: B-ORG -> O
Muudatus sõnes XXX: I-PER -> O
Muudatus sõnes Mis: B-PER -> O
Muudatus sõnes Tõnnis: I-PER -> B-PER
Muudatus sõnes Tõnnis: I-PER -> B-PER
Muudatus sõnes Rein: I-PER -> B-PER
Muudatus sõnes An: O -> B-PER
Muudatus sõnes kohut: I-ORG -> O
Muudatus sõnes kohtule: I-ORG -> O
Muudatus 

Muudatus sõnes -: I-LOC -> O
Muudatus sõnes -: I-LOC -> O
Muudatus sõnes H: O -> B-PER
Muudatus sõnes -: I-LOC -> O
Muudatus sõnes -: I-LOC -> O
Muudatus sõnes Kusta: I-PER -> B-PER
Muudatus sõnes -: I-LOC -> O
Muudatus sõnes Kusta: I-PER -> B-PER
Muudatus sõnes -: I-LOC -> O
Muudatus sõnes -: I-LOC -> O
Muudatus sõnes -: I-LOC -> O
Muudatus sõnes -: I-LOC -> O
Muudatus sõnes Abram: I-PER -> B-PER
Muudatus sõnes Iwan: I-PER -> B-PER
Muudatus sõnes Joseph: I-PER -> B-PER
Muudatus sõnes Kui: I-PER -> O
Muudatus sõnes A: O -> B-PER
Muudatus sõnes A: O -> B-PER
Muudatus sõnes A: O -> B-PER
Muudatus sõnes Adu: I-PER -> B-PER
Muudatus sõnes Ado: B-LOC_ORG -> B-PER
Muudatus sõnes Ado: B-LOC_ORG -> B-PER
Muudatus sõnes H: O -> B-PER
Muudatus sõnes Jaan: I-PER -> B-PER
Muudatus sõnes Härra: I-PER -> O
Muudatus sõnes Se: B-PER -> O
Muudatus sõnes Madli: B-LOC_ORG -> B-PER
Muudatus sõnes Summa: I-PER -> O
Tulemuste ammutamine on lõpetatud.
Muudatus sõnes Jürri: B-LOC_ORG -> B-PER
Muudatus sõnes )

Tulemuste ammutamine on lõpetatud.


---

In [20]:
df = dict()

for key in all_results.keys():
    results = results_by_subdistribution(all_results[key][0])
    precision = results[0]
    recall = results[1]
    f1 = results[2]
    df[key] = [precision, recall, f1]

In [23]:
display(pd.DataFrame(df, index=['Precision', 'Recall', 'F1']))

,"True, 85, 100","True, 90, 100","True, 95, 100","False, 85, 95","False, 85, 99","False, 90, 95","False, 90, 99","False, 95, 95","False, 95, 99"
Precision,0.906774,0.907746,0.908121,0.891467,0.895874,0.895049,0.899524,0.896809,0.901304
Recall,0.876615,0.877087,0.876786,0.873868,0.875842,0.874383,0.876400,0.873138,0.875156
F1,0.891440,0.892153,0.892179,0.882580,0.885745,0.884595,0.887812,0.884815,0.888037


The first `boolean` value states if `O` tags are completely left out or not.

The second value states the threshold for changing named entity labels from one to another:
e.g. if "Jaan" is 91% `B-PER` and 9% `I-PER` and if the threshold is `90`, all the `I-PER` tags
are also changed to `B-PER`. Should the threshold be for example 92, the named entity label 
stays unchanged.

The third value states the threshold for changing labels from or to `O` based on the threshold.